In [ ]:
#Install required packages first
pip install -r packages.txt

In [ ]:
import os
cwd = os.getcwd()
cwd

In [15]:
import pybel
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re

# Read files


In [16]:
#Orininal Data
nmrData = pd.read_csv('data/normalized_data/NMR_data_new.csv')

#Similarity Search on ChEMBL #Andrea's KNINE Analysis
nmr2chembl = pd.read_excel('data/normalized_data/Protein-Screening-Summary_CHEMBL30_Actives_teams_max08tan.xlsx')

#Similarity Search on PubChem #Andrea's KNINE Analysis
nmr2pchem = pd.read_excel('data/normalized_data/PubChem_Actives_Literature_NCATS.xlsx',sheet_name='Similar_065')


C:\Users\reagon.karki\Anaconda3\envs\bycovid\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [17]:
nmr2chembl.head(1)

,Z-ID,CID,Original Structure,Tanimoto,compared structure,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,Compound Key,...,Assay Parameters,Assay Variant Accession,Assay Variant Mutation,Target ChEMBL ID,Target Name,Target Organism,Target Type,Source Description,Document Journal,Document Year
0,Z2856434836,880785.0,NaN,0.649123,NaN,CHEMBL444186,RAZOXANE,4.0,268.27,Razoxane,...,NaN,NaN,NaN,CHEMBL4523582,Replicase polyprotein 1ab,Severe acute respiratory syndrome coronavirus 2,SINGLE PROTEIN,SARS-CoV-2 Screening Data 2020-21,NaN,2020.0


# Create an emtpy Graph 

In [18]:
nmrGraph = pybel.BELGraph(name='NMRgraph')

# Populate Graph

In [19]:
for i in range(len(nmrData)):
    nmrGraph.add_association(Abundance(namespace='Enamine',name=nmrData['source'][i]),Protein(namespace='VP', 
    name=nmrData['target'][i]),citation="CovidNMR",evidence='NMR_data')

In [20]:
for i in range(len(nmr2chembl)):
    nmrGraph.add_association(Abundance(namespace='Enamine',name=nmr2chembl['Z-ID'][i]),Pathology(namespace='ChEMBL',name=nmr2chembl['Molecule ChEMBL ID'][i]),citation='ChEMBL',evidence='Similarity search',Similarity = nmr2chembl['Tanimoto'][i],ChEMBLassay = nmr2chembl['Assay ChEMBL ID'],pChEMBL=nmr2chembl['pChEMBL Value'][i])
    nmrGraph.add_association(Pathology(namespace='ChEMBL',name=nmr2chembl['Molecule ChEMBL ID'][i]),MicroRna(namespace='HP',name=nmr2chembl['Target Name'][i]),citation='ChEMBL', evidence = 'Similarity search')

In [21]:
for i in range(len(nmr2pchem)):
    nmrGraph.add_association(Abundance(namespace='Enamine',name=nmr2pchem['Z-ID'][i]),Population(namespace='CID',name=str(nmr2pchem['cid'][i])),citation='Pubchem',evidence='Similarity search',Similarity = nmr2pchem['Tanimoto'][i],AID=str(nmr2pchem['aid'][i]))
    #nmrGraph.add_association(Pathology(namespace='ChEMBL',name=nmr2pchem['Molecule ChEMBL ID'][i]),MicroRna(namespace='HP',name=nmr2chembl['Target Name'][i]),citation='ChEMBL', evidence = 'Similarity search')

# Visualize Graph

In [22]:
to_jupyter(nmrGraph)

<IPython.core.display.Javascript object>

In [23]:
nmrGraph.summarize()

---------------------  --------
Name                   NMRgraph
Version
Number of Nodes        427
Number of Namespaces   5
Number of Edges        2222
Number of Annotations  0
Number of Citations    3
Number of Authors      0
Number of Components   1
Number of Warnings     0
Network Density        1.22E-02
---------------------  --------

Type (5)      Count  Example
----------  -------  ------------------------------------
Abundance       283  a(Enamine:Z1271660837)
Pathology        77  path(ChEMBL:CHEMBL1917204)
Population       36  pop(CID:60149)
Protein          24  p(VP:"Nsp5 WT")
miRNA             7  m(HP:"Alpha-2a adrenergic receptor")

Prefix (5)    Name                        Count  Example
------------  ------------------------  -------  ------------------------------------
Enamine                                     283  a(Enamine:Z373221060)
ChEMBL        ChEMBL                         77  path(ChEMBL:CHEMBL354349)
CID           PubChem CID                    36  pop(CID:7

In [24]:
chemList = list(nmr2chembl['Molecule ChEMBL ID'])
uniqChemList = list(set(chemList))
len(uniqChemList)

77

In [25]:
def RetMech(chemblIds):
    getMech = new_client.mechanism
    mechList = []
    for i in range(len(chemblIds)):
        mechs = getMech.filter(molecule_chembl_id=chemblIds[i]).only(['mechanism_of_action','target_chembl_id'])
        #mechs = getMech.filter(molecule_chembl_id=chemblIds[i])
        print(mechs)
        mechList.append(list(mechs))
    named_mechList = dict(zip(chemblIds,mechList))
    named_mechList = {k: v for k, v in named_mechList.items() if v}
    return(named_mechList)


In [26]:
mechList = RetMech(uniqChemList)


[{'mechanism_of_action': 'Histamine H1 receptor antagonist', 'target_chembl_id': 'CHEMBL231'}]
[]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'Glycogen synthase kinase-3 beta inhibitor', 'target_chembl_id': 'CHEMBL262'}]
[]
[]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'DNA topoisomerase II alpha inhibitor', 'target_chembl_id': 'CHEMBL1806'}]
[]
[]
[]
[{'mechanism_of_action': 'D2-like dopamine receptor agonist', 'target_chembl_id': 'CHEMBL2331075'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'Unknown', 'target_chembl_id': None}]
[{'mechanism_of_action': 'DNA disrupting agent', 'target_chembl_id': 'CHEMBL2311221'}]
[]
[]
[]
[{'mechanism_of_action': 'Bacterial dihydropteroate synthase inhibitor', 'target_chembl_id': 'CHEMBL2364668'}]
[]
[]
[{'mechanism_of_action': 'DOPA decarboxylase inhibitor', 'target_chembl_id': 'CHEMBL1843'}]
[]
[]
[{'mechanism_of_action': 'Thioredoxin inhibitor', 'target_chembl_id': 'CHEMBL2010624'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'mechani

In [27]:
for i in mechList:
    #print(i)
    #break
    #print(named_mechList[i])
    #print(len(named_mechList[i]))
    #break
    for j in range(len(mechList[i])):
        #print(named_mechList[i][j]['mechanism_of_action'])
        #print(named_mechList[i][j]['target_chembl_id'])
        #print(i)
        #break
        nmrGraph.add_association(Pathology(namespace='ChEMBL',name=i),BiologicalProcess(namespace='MOA',name=mechList[i][j]['mechanism_of_action']),citation='ChEMBL database',evidence='ChEMBL query')
        if not mechList[i][j]['target_chembl_id'] == None:
            nmrGraph.add_association(Pathology(namespace='ChEMBL',name=i),MicroRna(namespace='HP',name=mechList[i][j]['target_chembl_id']),citation='ChEMBL database',evidence='ChEMBL query')


In [28]:
def RetAct(chemblIds,j):
    GetAct = new_client.activity
    ActList = []
    #for i in range(len(chemblIds)):
    #print(chemblIds[0])
    for i in range(len(chemblIds)):
        filtered_list=['assay_chembl_id','assay_type','pchembl_value','target_chembl_id','target_organism','bao_label','target_pref_name']
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,assay_type_iregex='(B|F)',target_organism='Homo sapiens').only(filtered_list)
        acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,target_organism='Homo sapiens').only(filtered_list)
        j=j+1
        #acts = [d for d in acts if d.get('target_organism') == 'Homo sapiens']
        #acts = [d for d in acts if d.get('bao_label') == 'single protein format']
        #acts = [d for d in acts if d.get('type') in ['Ki', 'IC50']]
        acts = [d for d in acts if float(d.get('pchembl_value')) >= 6]
        print(j)
        #print(len(acts))
        acts = acts[:5]   
        #print(acts)
        ActList.append(list(acts))
    #print(ActList)
    named_ActList = dict(zip(chemblIds,ActList))
    named_ActList = {k: v for k, v in named_ActList.items() if v}
    return(named_ActList)


In [30]:
ActList = RetAct(uniqChemList,0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77


In [31]:
for i in ActList:
    for j in range(len(ActList[i])):
        #print(named_mechList[i][j]['mechanism_of_action'])
        #print(named_mechList[i][j]['target_chembl_id'])
        #print(i)
        #break
        #nmrGraph.add_association(Pathology(namespace='ChEMBL',name=i),BiologicalProcess(namespace='MOA',name=mechList[i][j]['mechanism_of_action']),citation='ChEMBL database',evidence='ChEMBL query')
        if not ActList[i][j]['target_chembl_id'] == None:
            nmrGraph.add_association(Pathology(namespace='ChEMBL',name=i),MicroRna(namespace='HP',name=ActList[i][j]['target_pref_name']),citation='ChEMBL database',evidence='ChEMBL query')


In [32]:
def RetDrugInd(chemblIDs):
    getDrugInd = new_client.drug_indication
    drugIndList = []
    for i in range(len(chemblIDs)):
        drugInd = getDrugInd.filter(molecule_chembl_id=chemblIDs[i]).only('mesh_heading')
        #drugInd = getDrugInd.filter(molecule_chembl_id=chemblIDs[i])
        print(drugInd)
        drugIndList.append(list(drugInd))
    named_drugIndList = dict(zip(chemblIDs,drugIndList))
    named_drugIndList = {k: v for k, v in named_drugIndList.items() if v}
    return(named_drugIndList)

In [33]:
drugIndList = RetDrugInd(uniqChemList)

[{'mesh_heading': 'Hypersensitivity'}]
[{'mesh_heading': 'Esophagitis, Peptic'}]
[]
[{'mesh_heading': 'Diabetes Mellitus, Type 2'}]
[]
[]
[]
[{'mesh_heading': 'Alzheimer Disease'}, {'mesh_heading': 'Child Development Disorders, Pervasive'}, {'mesh_heading': 'Myotonic Dystrophy'}]
[]
[]
[]
[]
[{'mesh_heading': 'Non-alcoholic Fatty Liver Disease'}, {'mesh_heading': 'Liver Cirrhosis'}, {'mesh_heading': 'Lung Neoplasms'}]
[]
[{'mesh_heading': 'Tuberculosis'}, {'mesh_heading': 'Tuberculosis, Pulmonary'}, {'mesh_heading': 'Liver Diseases'}, {'mesh_heading': 'Leprosy'}, '...(remaining elements truncated)...']
[{'mesh_heading': 'Glioblastoma'}, {'mesh_heading': 'Neoplasms'}, {'mesh_heading': 'Small Cell Lung Carcinoma'}, {'mesh_heading': 'Carcinoma, Pancreatic Ductal'}, '...(remaining elements truncated)...']
[]
[]
[]
[{'mesh_heading': 'Parkinson Disease'}, {'mesh_heading': 'Neoplasms'}, {'mesh_heading': 'Osteoporosis'}]
[]
[]
[]
[]
[]
[]
[{'mesh_heading': 'Neoplasms'}]
[]
[]
[]
[]
[]
[]
[{'me

In [34]:

for i in drugIndList:
    # print(i)
    # break
    # print(named_drugIndList[i])
    # print(len(named_drugIndList[i]))
    # break
    for j in range(len(drugIndList[i])):
        # print(named_drugIndList[i][j]['mesh_heading'])
        # print(i)
        # break
        nmrGraph.add_association(Pathology(namespace='ChEMBL', name=i),
                                 Population(namespace='Disease', name=drugIndList[i][j]['mesh_heading']),
                                 citation='ChEMBL database', evidence='ChEMBL query')

In [35]:
to_jupyter(nmrGraph)

<IPython.core.display.Javascript object>

In [36]:
nmrGraph.summarize()

---------------------  --------
Name                   NMRgraph
Version
Number of Nodes        674
Number of Namespaces   7
Number of Edges        2896
Number of Annotations  0
Number of Citations    4
Number of Authors      0
Number of Components   1
Number of Warnings     0
Network Density        6.38E-03
---------------------  --------

Type (6)             Count  Example
-----------------  -------  ----------------------------------------
Abundance              283  a(Enamine:Z1891773393)
Population             177  pop(Disease:Angioedema)
miRNA                   99  m(HP:"6-phosphogluconate dehydrogenase")
Pathology               77  path(ChEMBL:CHEMBL929)
Protein                 24  p(VP:"ORF9a (CTD)")
BiologicalProcess       14  bp(MOA:Unknown)

Prefix (7)    Name                        Count  Example
------------  ------------------------  -------  -----------------------------------
Enamine                                     283  a(Enamine:Z33545544)
Disease                  

# Function to filter graph on selective viral proteins

In [37]:
nsp5_prot = ['_GHM_nsp5','_GS_nsp5','Nsp5 WT']
nsp3_prot = ['nsp3a','nsp3b','nsp3b·GS-441524','nsp3c (SUD-MC)','nsp3c (SUD-N)','nsp3d','nsp3e','nsp3y']

def filter_graph(mainGraph,vprotList):
    nsp_list = []
    for u, v, data in mainGraph.edges(data=True): 
        if u.name in vprotList or v.name in vprotList: 
            nsp_list.append(u)
            nsp_list.append(v)
        
    for u, v, data in mainGraph.edges(data=True): 
        if 'Similarity' not in data: 
            continue 
        if data['Similarity'] >= .65 and (u in nsp_list or v in nsp_list): 
            #yield u, v, k, data
            nsp_list.append(u)
            nsp_list.append(v)
            
    #print(nsp_list)
    
    nsp_graph = mainGraph.subgraph(nsp_list)
    #return(to_jupyter(nsp_graph))
    final_nsp = [node for node in nsp_graph.nodes() if isinstance(node,pybel.dsl.Protein) and node.name in vprotList]
    #print(final_nsp)
    for u, v, data in nsp_graph.edges(data=True): 
        if u.namespace in ['Enamine','ChEMBL','CID'] and v.namespace in ['Enamine','ChEMBL','CID']:
            final_nsp.append(u)
            final_nsp.append(v)
    
    #return(nsp_graph.subgraph(final_nsp))
    nsp_graph = nsp_graph.subgraph(final_nsp)
    #chembl_act = []
    nsp_nodes = [node for node in nsp_graph.nodes()]
    for u in nsp_graph.nodes():
        if u.namespace == 'ChEMBL':
            #test = nmrGraph.neighbors(u)
            chem = [n for n in mainGraph.neighbors(u)]
            nsp_nodes.extend(chem)
            
    return(mainGraph.subgraph(nsp_nodes))

    
    

# Sub-graph for nsp5 proteins

In [38]:
nsp5Graph = filter_graph(nmrGraph,nsp5_prot)
to_jupyter(nsp5Graph)

<IPython.core.display.Javascript object>

# Sub-graph for nsp3 proteins

In [39]:
nsp3Graph = filter_graph(nmrGraph,nsp3_prot)
to_jupyter(nsp3Graph)

<IPython.core.display.Javascript object>

In [32]:
pybel.to_graphml(nmrGraph,'data/NMRGraph.graphml')